<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
'''Importing necessary modules
'''
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('./Trainers/')
sys.path.append('./Dataloaders/')
sys.path.append('./utils/')

from dataloader import mydataset, create_prime_dict 
'''
For BERT
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from trainer_BERT import train, test_classify

In [ ]:
# Select gpu
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:7


**Dataloading Scheme**

In [ ]:
trainlist = 'add path of training list'
validlist = 'add path of validation list'

In [ ]:
prime_dict = create_prime_dict(trainlist)

Number of classes =  2


In [ ]:
'''
Train Dataloader
''' 
train_dataset = mydataset(trainlist, prime_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 32, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mydataset(validlist, prime_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 32, num_workers=16,pin_memory=True)

**Model Definition**

In [ ]:
'''
Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", #12-layer BERT model, with an uncased vocab.
    num_labels = 2, #Binary classification
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model = nn.DataParallel(model,device_ids=[7]).to(device)
model

**Hyperparameters**

In [ ]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()


'''
Optimizer
'''
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

'''
Number of training epochs. The BERT authors recommend between 2 and 4. We chose to run for 4, but we'll see later that this may be over-fitting the training data.
'''
num_Epochs = 10


'''
Create the learning rate scheduler.
Total number of training steps is [number of batches] x [number of epochs].
Note that this is not the same as the number of training samples.
'''
total_steps = len(train_dataloader) * num_Epochs
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,  num_training_steps = total_steps)

In [ ]:
modelname = 'BERT_basic'
modelpath = './saved_model_checkpoints/'+modelname

In [ ]:
writer = SummaryWriter(modelname)

train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

In [ ]:
'''
Load saved model from checkpoint  #####
'''
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


In [ ]:
model.module.classifier

Linear(in_features=768, out_features=2, bias=True)